In [67]:
import numpy as np
import time
import json
from datetime import datetime
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from ultralytics import YOLO
import os

Поменяем местами названия папок test,val местами. Сделаем это ради удобства обучения модели. Сами данные никак не меняются, меняется только название папок

# Построение пайплайна

Создадим пайплайн по обучению моделей классификации. В нашем случае предоставим выбор пользователю из семейства моделей YOLO.

In [99]:
import json
import time
from datetime import datetime
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from ultralytics import YOLO  # Предполагается, что вы используете библиотеку ultralytics

def model_cv(model_name, **kwargs):
    # Загрузка модели в зависимости от имени
    if model_name == 'nano':
        model = YOLO('/home/c4/Рабочий стол/C4_M3/yolo11n-cls.pt')
    elif model_name == 'small':
        model = YOLO('/home/c4/Рабочий стол/C4_M3/yolo11s-cls.pt')
    elif model_name == 'mean':
        model = YOLO('/home/c4/Рабочий стол/C4_M3/yolo11m-cls.pt')
    else:
        raise ValueError("Недопустимое имя модели. Выберите 'nano', 'small' или 'mean'.")

    # Запуск обучения и измерение времени
    start_time = time.time()
    
    # Обучение модели
    res = model.train(data='/home/c4/Рабочий стол/C4_M2/preprocessed_images/', **kwargs)

    # Оценка времени на обучение
    total_training_time = time.time() - start_time
    training_time_per_epoch_sec = total_training_time / kwargs.get('epochs', 1)  # Делим на количество эпох
    training_time_min = total_training_time / 60  # Время в минутах

    # Валидация модели
    results = model.val()
    #Расчет метрик
    y_true = []
    y_pred = []
    y_pred_prob = []
    for dirpath, dirnames, filenames in os.walk('/home/c4/Рабочий стол/C4_M2/preprocessed_images/test'):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            y_true.append(list(map(lambda x: int(x), file_path.split('/')[7].split('-')))[1])
            result = model.predict(file_path)
            for r in result:
                a = r.probs.top1.numpy()
                b = r.probs.data.numpy()
                y_pred_prob.append(b)
                y_pred.append(a)
        
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        y_pred_prob = np.array(y_pred_prob)
        print('y_pred_prob', y_pred_prob)

        # Расчет метрик
        roc_auc = roc_auc_score(y_true, y_pred_prob, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')

        print(f"ROC AUC: {roc_auc:.3f}")
        print(f"Accuracy: {accuracy:.3f}")
        print(f"F1 Score (macro): {f1:.3f}")
        
        print(f"Время на одну эпоху (сек.): {training_time_per_epoch_sec:.3f}")
        print(f"Общее время обучения (мин.): {training_time_min:.3f}")

        # Сохранение результатов в файл с текущей датой и временем
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        results_filename = f"model_results_{timestamp}.json"

        results_data = {
            'roc_auc': roc_auc,
            'accuracy': accuracy,
            'f1_score': f1,
            'training_time_per_epoch_sec': training_time_per_epoch_sec,
            'training_time_min': training_time_min,
            'timestamp': timestamp,
            'model_name': model_name,
            'val_results': results,
            'train_results': res,
        }

        with open(results_filename, 'w') as f:
            json.dump(results_data, f)

        return [model, results, results_data]
    
    else:
        print("Результаты валидации не содержат предсказаний.")
        return None



# Обучение модели и оценка точности

In [100]:
res = model_cv('nano', 
            imgsz=224, epochs=1,
            device='cpu', seed=41,
         hsv_v = 0.1, cache = True, exist_ok=True, time=1/240
        )

Ultralytics 8.3.70 🚀 Python-3.10.16 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=classify, mode=train, model=/home/c4/Рабочий стол/C4_M3/yolo11n-cls.pt, data=/home/c4/Рабочий стол/C4_M2/preprocessed_images/, epochs=1, time=0.004166666666666667, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=True, device=cpu, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=41, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

train: Scanning /home/c4/Рабочий стол/C4_M2/preprocessed_images/train... 23004 images, 0 corrupt: 100%|███████

WARNING ⚠️ Classification `cache_ram` training has known memory leak in https://github.com/ultralytics/ultralytics/issues/9824, setting `cache_ram=False`.



val: Scanning /home/c4/Рабочий стол/C4_M2/preprocessed_images/val... 1000 images, 0 corrupt: 100%|██████████| 

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=1e-05, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/classify/train
Starting training for 0.004166666666666667 hours...

      Epoch    GPU_mem       loss  Instances       Size


...

               classes   top1_acc   top5_acc: 100%|██████████| 32/32 [00:14<00:00,  2.15it/s]


                   all      0.002      0.008
Speed: 0.0ms preprocess, 3.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Ultralytics 8.3.70 🚀 Python-3.10.16 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
YOLO11n-cls summary (fused): 112 layers, 2,807,024 parameters, 0 gradients, 4.2 GFLOPs
train: /home/c4/Рабочий стол/C4_M2/preprocessed_images/train... found 23004 images in 1000 classes ✅ 
val: /home/c4/Рабочий стол/C4_M2/preprocessed_images/val... found 1000 images in 1000 classes ✅ 
test: /home/c4/Рабочий стол/C4_M2/preprocessed_images/test... found 2000 images in 1000 classes ✅ 
WARNING ⚠️ Classification `cache_ram` training has known memory leak in https://github.com/ultralytics/ultralytics/issues/9824, setting `cache_ram=False`.


val: Scanning /home/c4/Рабочий стол/C4_M2/preprocessed_images/val... 1000 images, 0 corrupt: 100%|██████████| 
               classes   top1_acc   top5_acc: 100%|██████████| 63/63 [00:17<00:00,  3.62it/s]


                   all      0.002      0.008
Speed: 0.0ms preprocess, 3.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
y_pred_prob []


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.